# 📘 MNIST Curve Adjustment Layers (CALs) — Experimental Baseline

## 🧠 Overview

In this notebook, we establish a **strong and interpretable control model** on the **MNIST handwritten digit classification task**, forming the **foundation for our exploration of Curve Adjustment Layers (CALs)** — a novel technique aimed at improving neural network convergence and generalization.

CALs act as *intermediary modules* between neural layers, applying **learned deformation curves** to activations. These curves serve to *bend the output space* toward more optimal configurations, enabling the model to adaptively reshape internal representations.

This notebook provides:
- ✅ A solid CNN-based reference model
- 📥 MNIST dataset loading and visualization
- 🧰 Preliminaries for future CAL insertion and ablation studies

---


## Install any packages needed ...

In [15]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/106.6 kB ? eta -:--:--
     ------- ----------------------------- 20.5/106.6 kB 682.7 kB/s eta 0:00:01
     ----------------------------- --------- 81.9/106.6 kB 1.2 MB/s eta 0:00:01
     -------------------------------------- 106.6/106.6 kB 1.2 MB/s eta 0:00:00
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
    --------------------------------------- 0.2/8.1 MB 3.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.5/8.1 MB 5.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.1/8.1 MB 7.8 MB/s eta 0:00:01
   ------ --------------------------------- 1.2/8.1 MB 7.2 MB/s eta 0:00:01
   ------ --------------------------------- 1.4/8.1 MB 5.8 MB/s eta 0:00:02
   ---------- ----------------------------- 2.1/8.1 MB 7.4 MB/s eta 0:00:01
   ------------ ----------------

In [ ]:
import os
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

# Grab the MNIST Dataset:
---

The MNIST dataset is a collection of 70,000 handwritten digits (0-9) that is commonly used for training various image processing systems.


## 📊 Dataset Summary: MNIST

- **Type**: Grayscale handwritten digits  
- **Size**: `60,000` training / `10,000` test samples  
- **Resolution**: `28×28` pixels  
- **Classes**: Digits `0–9`  

---

In [ ]:
# Download MNIST dataset to a local directory:
# M:\dev\ml\data

data_path = os.path.join('..', 'data')
mnist_train = datasets.MNIST( data_path, train=True, download=True )
mnist_test  = datasets.MNIST( data_path, train=False, download=True )
print( "MNIST dataset downloaded to:", data_path )

count_train = len( mnist_train )
count_test = len( mnist_test )

MNIST dataset downloaded to: ..\data


# Examine the data:
---

Read data samples and view the image and label ...

In [ ]:
import torch
print( "Number of training samples:", count_train )
print( "Number of test samples:", count_test )

# Examine data samples:
img, label = mnist_train[ torch.randint( 0, count_train, ( ) ) ]
print( "Image:", img, "Label:", label )

# Display the first image:
img.show( )

Number of training samples: 60000
Number of test samples: 10000
Image: <PIL.Image.Image image mode=L size=28x28 at 0x1627A6B94C0> Label: 1


## 📦 Data Loader & Hyperparameter Setup

To train our CNN model efficiently, we define a few key **training hyperparameters** and configure a **data preprocessing pipeline** that prepares the MNIST dataset for use in PyTorch.

---

### 🎛️ Training Hyperparameters

| Hyperparameter   | Value      | Description                                                       |
|------------------|------------|-------------------------------------------------------------------|
| `batch_size`     | `64`       | Number of samples per batch — balances speed and gradient quality |
| `epochs`         | `5`        | Total passes over the training set                               |
| `lr`             | `0.001`    | Learning rate — controls optimizer step size                      |

---


In [ ]:
# Hyperparameters
batch_size = 64
epochs = 5
lr = 0.001

# Create the transform function to convert the image to a tensor and normalize it:
# (Note: The normalization values are based on the MNIST dataset statistics.)
transforms = transforms.Compose( [
    transforms.ToTensor( ),
    transforms.Normalize( ( 0.1307, ), ( 0.3081, ) ),
] )

# Create a DataLoader to load the dataset in batches:
data_loader = DataLoader( mnist_train,
                          batch_size=batch_size,
                          shuffle=True,
                          transform=transforms )



# 🧠 MNIST Control Model Architecture Overview

## 🎯 Objective

This notebook establishes a **well-performing, yet minimal and interpretable convolutional neural network (CNN)** designed to solve the **MNIST handwritten digit classification problem**. Our intent is to create a **robust "control" architecture** for later comparative experiments involving more advanced or experimental models. MNIST is a widely used benchmark dataset for evaluating model performance in image recognition tasks.

---


## 🧱 Model Architecture

A lightweight yet competitive CNN inspired by classical deep learning pipelines.

| Layer              | Type            | Shape In → Out      | Activation | Notes                         |
|--------------------|------------------|----------------------|------------|-------------------------------|
| 🔹 Input           | Image            | `1 × 28 × 28`        | –          | Grayscale digit image         |
| 🔸 Conv1           | Conv2d(1→32, 3×3) | `28 × 28 → 28 × 28`  | ReLU       | Padding preserves resolution  |
| 🔸 MaxPool1        | MaxPool2d(2×2)   | `28 × 28 → 14 × 14`  | –          | Downsamples feature maps      |
| 🔸 Conv2           | Conv2d(32→64, 3×3)| `14 × 14 → 14 × 14` | ReLU       | Captures mid-level patterns   |
| 🔸 MaxPool2        | MaxPool2d(2×2)   | `14 × 14 → 7 × 7`    | –          | Further spatial reduction     |
| 🔸 Flatten         | –                | `64 × 7 × 7 → 3136`  | –          | Prep for dense layers         |
| 🔸 FC1             | Linear(3136→128) | `3136 → 128`         | ReLU       | Dense representation          |
| 🔸 Dropout         | Dropout(p=0.25)  | `128 → 128`          | –          | Regularization against overfit|
| 🔸 FC2             | Linear(128→10)   | `128 → 10`           | –          | Raw class logits (pre-Softmax)|

---

## ⚙️ Design Rationale

### ✅ Simplicity with Performance
This architecture balances **simplicity**, **speed**, and **performance**, achieving **~99% accuracy** on MNIST while remaining **transparent and modifiable**.

### 🧩 Key Design Choices

- **2× Convolution Layers**: Enough for capturing local and mid-level features on 28×28 inputs.
- **MaxPooling**: Reduces computation, introduces translational invariance.
- **Dropout**: Prevents overfitting in dense layers, especially for small datasets.
- **ReLU Activations**: Promote fast convergence and sparse gradients.
- **Minimal FC layers**: Keeps parameter count low without sacrificing accuracy.

---

## 🧪 Purpose in Research

This model acts as a **baseline control** for later experiments involving:

- Architectural modifications (residuals, batch norm, transformers, etc.)
- Alternative optimization strategies (SGD vs AdamW)
- Ablation studies
- Regularization and generalization research

It serves as a **trustworthy metric anchor** to assess whether newer approaches offer real improvements or are overly complex.

---

## 📈 Performance Baseline

With standard hyperparameters:

```python
epochs = 5
batch_size = 64
optimizer = Adam(lr=1e-3)
loss_fn = CrossEntropyLoss()


In [ ]:
# Define a convolutional neural network (CNN) model
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model
class MNISTNet( nn.Module ):
    def __init__( self ):
        """
        Initialize the CNN model.
        """
        
        super( ).__init__( )
        
        # Define the layers of the CNN:
        self.conv1 = nn.Conv2d( 1, 32, 3, padding=1 )
        self.conv2 = nn.Conv2d( 32, 64, 3, padding=1 )
        self.pool = nn.MaxPool2d( 2, 2 )
        self.fc1 = nn.Linear( 64 * 7 * 7, 128 )
        self.dropout = nn.Dropout( 0.25 )
        self.fc2 = nn.Linear( 128, 10 )

    def forward( self, x ):
        """
        Define the forward pass of the model.
        Args:
            x (torch.Tensor): Input tensor.
        Returns:
            torch.Tensor: Output tensor.
        """
        # Apply the convolutional layers, activation functions, and pooling:
        # (Note: The input tensor is expected to have shape [batch_size, 1, 28, 28])
        x = self.pool( torch.relu( self.conv1(x) ) )
        x = self.pool( torch.relu( self.conv2(x) ) )
        x = torch.flatten( x, 1 )
        x = self.dropout( torch.relu( self.fc1(x) ) )
        x = self.fc2( x )
        return x
